In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

def save_image_from_array(array, output_path, size=(28, 28)):
    reshaped = array.reshape(size)
    plt.imsave(output_path, reshaped, cmap='gray')

def convert_npy_to_images(npy_path, output_dir, limit=1000, size=(28, 28)):
    os.makedirs(output_dir, exist_ok=True)
    data = np.load(npy_path, allow_pickle=True)
    for i, img_array in tqdm(enumerate(data[:limit]), total=limit):
        save_image_from_array(img_array, os.path.join(output_dir, f"{i}.png"), size)

categories = ["apple", "cloud", "flower", "leaf"]

for category in categories:
    convert_npy_to_images(f"{category}.npy", f"dataset/{category}", limit=1000, size=(28, 28))  # adjust if not 28x28


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 221.79it/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Data Generator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% train, 20% val
)

train_generator = datagen.flow_from_directory(
    'dataset',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'dataset',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Save the model
model.save('model.h5')


Found 3200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 548ms/step - accuracy: 0.7522 - loss: 0.7094 - val_accuracy: 0.8675 - val_loss: 0.3419
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - accuracy: 0.8948 - loss: 0.3005 - val_accuracy: 0.9187 - val_loss: 0.2569
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9307 - loss: 0.2036